## 1. Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

## 2. Carregando dados

In [ ]:
actors = pd.read_csv('dados/cast_data.csv')
link_data = pd.read_csv('dados/cast_and_movies_data.csv')

In [ ]:
actors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044499 entries, 0 to 1044498
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   id      1044499 non-null  int64  
 1   name    1044499 non-null  object 
 2   birth   168351 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 23.9+ MB


In [ ]:
link_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189594 entries, 0 to 1189593
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   person_id  1189594 non-null  int64
 1   movie_id   1189594 non-null  int64
dtypes: int64(2)
memory usage: 18.2 MB


## 3. Algoritmo

**Objetivo**: encontrar o grau de Bacon dado um ator/atriz.

O problema pode ser descrito como um grafo, onde os nós são os atores e as arestas unem dois atores/atrizes que atuaram juntos em um filme qualquer. Vejamos um exemplo, digamos que $a$, $b$, $c$, $d$, $e$, $f$ e $g$ representam os atores, como no diagrama abaixo:

<img width=400 src="https://user-images.githubusercontent.com/56659549/159192137-5dbcc9af-126b-4de6-acd1-286631a1d389.jpg">

Podemos perceber que o ator $b$, por exemplo, atuou com os atores $a$, $d$ e $f$ em um filme qualquer. Enquanto os atores $c$ e $d$ nunca atuaram juntos até então.

Estamos interessados em medir o Grau de Bacon entre dois atores/atrizes, ou seja, a "distância" entre dois nós do grafo. Por exemplo, no grafo acima, o Grau de Bacon $gb(a, b)$ = 0, ou seja, a "distância" entre $a$ e $b$ é 0, enquanto $gb(c, f) = 1$. Note que, para quaisquer atores $x$ e $y$, $gb(x, y) = gb(y, x)$. Desenvolveremos a seguir um algoritmo capaz de computar o Grau de Bacon entre quaisquer atores/atrizes $x$ e $y$.

Para cada ator/atriz $x$, o conjunto $A_x$ representa todos os artistas que estão ligados diretamente com $x$. No exemplo acima, teríamos

$A_a = \{b, c\};$ <br>
$A_b = \{a, d, f\};$<br>
$A_c = \{a, e\};$<br>
$A_d = \{b\};$<br>
$A_e = \{c, f\};$<br>
$A_f = \{b, e, g\}.$<br>
$A_g = \{f\}.$<br>

Ainda, seja $N_k$ o *network*, ou seja, o conjunto de todos os atores/atrizes cujo Grau de Bacon em relação a $x$ é menor ou igual a $k$. Para o exemplo acima, se tomarmos $x=a$, teremos

$N_0 = \{b, c\};$ <br>
$N_1 = \{b, c, d, e, f\};$<br>
$N_2 = \{b, c, d, e, f, g\};$<br>

Observemos que 

$k = 0 \rightarrow N_0 = A_a$<br>
$k = 1 \rightarrow N_1 = A_b \cup A_c$ <br>
$k = 2 \rightarrow N_2 = A_b \cup A_c \cup A_d \cup A_e$

De modo geral, para qualquer $x$, $N_k$ pode ser igualmente descrito como a união de todos os $A_i$, $i \in N_{k-1}$, ou seja,

$$N_k=\bigcup_{i \in N_{k-1}}A_i$$  

A vantagem de definirmos o $N_k$ como união dos $A_i$ é que podemos utilizar um algoritmo iterativo que depende de $N_{k-1}$. Deste modo, o algoritmo repetirá os passos de união dos conjuntos demonstrado acima até tentar encontrar o outro ator\atriz $y$ no conjunto $N_k$. Em outras palavras,

$$y \in N_k=\bigcup_{i \in N_{k-1}}A_i$$ 

Para elucidar o algoritmo, abaixo faremos um exemplo de seu funcionamento:

$x = a$ <br>
$y = g$ <br>
$N_0 = A_a = \{b, c\}$ <br>

$
k = 0: g \notin N_0 \rightarrow N_1 = A_b \cup A_c = \{b, c, d, e, f\}
$
<br>
$
k = 1: g \notin N_1 \rightarrow N_2 = A_b \cup A_c \cup A_d \cup A_e \cup A_f = \{b, c, d, e, f, g\} 
$
<br>
$
k = 2: g \in N_2 \rightarrow k = 2 
$
<br>
<br>
Como vimos acima, $g \in N$ apenas na 2 iteração, logo $gb(a, g)=2$, ou seja, o Grau de Bacon entre $a$ e $g$ é 2.

Abaixo temos o pseudocódigo do algoritmo implementado:

```
Algoritmo gb(x, y):
  Se x = y: 
    retorna 0

  # definindo N-K e N_(k-1)
  N0 = A[x]
  N1 = A[x]

  Para k em 0, 1, 2, ...:
    Se y em N1:
      retorna k
    Para i em N0:
      N1 = N1 + A[i]
    N0 = N1

  retorna 'Não encontrado'
```


In [ ]:
# relacionando todos os atores que interpretaram juntos em algum filme
merged = pd.merge(link_data, link_data, on = 'movie_id').query('person_id_x != person_id_y')

# criando os conjuntos de relações, assim como foi definido no algoritmo acima
# este representa os conjuntos A_i
A = merged.groupby('person_id_x').person_id_y.apply(set)

In [ ]:
def get_actor_id(actor_name):

    """
    tomar o id do ator pelo seu nome
    """

    return actors.query(f'name == "{actor_name}"').id.iloc[0]

def entry_validation(id_name):
    '''
    verifica se a id_name é um inteiro (representando o id do ator/atriz) ou se
    é o nome do ator/atriz, retornando assim seu respectivo id
    '''

    if not str(id_name).isdigit(): return get_actor_id(id_name)
    else: return id_name

def search_bacon_degree(x, y = 'Kevin Bacon', max_iter = 15):

    '''
    função para encontrar o Grau de Bacon entre dois atores. Por padrão, 
    o parâmetro y será "Kevin Bacon", indicando assim o Grau de Bacon do
    do ator fornecido em x. Esta função é equivalente ao gb(x, y).

    x -> str ou int
      id ou nome do primeiro ator/atriz

    y -> str ou int, default 'Kevin Bacon'
      id ou nome do segundo ator/atriz

    max_iter -> int, default 15
      número máximo de iterações de busca 

    A função retornará valores inteiros que vão de 0 a max_iter, onde:
      0 -> indica que atuaram juntos em um mesmo filme
      1 -> indica que o ator/atriz inicial atuou com alguém que atuou com
           o ator/atriz final
      2 -> indica que o ator/atriz inicial atuou com alguém que atuou com outro
           alguém que atuou com o ator/atriz final
      ...

    '''

    # verifica se x e y são o mesmo artista
    if x == y: return 0

    # validando entradas x e y, aceitam id's ou nomes
    x = entry_validation(x)
    y = entry_validation(y)

    # assumindo N0 = A_x, onde N0 representa N_(k-1) e N1 representa N_k
    # no início, N_k = N_(k-1). Foi necessário separa em dois set pois
    # não é possível iterar e incrementar o iterator utilizado no mesmo 
    # loop for
    N0 = A[x]
    N1 = N0.copy()

    for k in range(max_iter):

        # verifica se y está contido em N_k
        if y in N1: return k

        # caso o ator final não tenha sido encontrado e N_k, então N0 passa a ser 
        # o N1 e N1 é atualizado para a próxima iteração
        for j in N0:
            N1.update(A[j])

        # atualizando o N0
        N0 = N1.copy()
    
    # caso chegado na iteração máxima, é retornado que não foi encotrado
    # o Grau de Bacon
    return 'Não encontrado'

In [ ]:
search_bacon_degree('Ben Affleck', 'Kevin Bacon')

0

## 4. Observações
*    Não foram todos os atores listados que foram linkados em filmes;
*    Há muitos nomes repetidos com id's diferentes, investigar se há algum erro ou se são de fato pessoas diferentes com mesmo nome;
*    Há casos vários registros com mesmo nome, id diferente e uma das datas de aniversário preenchidas;
*    O site do oracleofbacon.org pode estar desatualizado ou com algum problema. Há casos como de atores que atuaram junto com Kevin Bacon (102) em um filme, mas com Grau de Bacon 2 no site. Isso ocorre com o ator Ben Affleck (255), por exemplo, que tem grau 2 no site mas que atuou junto a Kevin Bacon em Sundance Skippy (1518293).
